
# EDA de Pagos Reales desde Supabase - Proyecto Gym Master

Este notebook analiza los pagos reales extraídos de Supabase, calcula el perfil de pago de cada socio y explora la distribución de pagos, retrasos y métodos de pago.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuraciones generales
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:

# Cargar dataset real
pagos_df = pd.read_csv('../../Data_Lake_CSV/pagos_supabase.csv')

# Vista general del dataset
pagos_df.head()


In [ ]:

# --- Enriquecer con perfil de pago ---

# Calcular promedio de dias_retraso por socio
promedio_retraso = pagos_df.groupby('socio_id')['dias_retraso'].mean().reset_index()
promedio_retraso.rename(columns={'dias_retraso': 'promedio_dias_retraso'}, inplace=True)

# Asignar perfil_pago
def asignar_perfil(promedio):
    if promedio <= 1:
        return 'puntual'
    elif promedio <= 7:
        return 'leve_retraso'
    else:
        return 'moroso'

promedio_retraso['perfil_pago'] = promedio_retraso['promedio_dias_retraso'].apply(asignar_perfil)

# Unir al dataframe original
pagos_df = pagos_df.merge(promedio_retraso[['socio_id', 'perfil_pago']], on='socio_id')
pagos_df.head()


In [ ]:

# --- Distribución de pagos por mes y día ---
pagos_df['fecha_pago'] = pd.to_datetime(pagos_df['fecha_pago'])
pagos_df['mes_pago'] = pagos_df['fecha_pago'].dt.to_period('M')
pagos_df['dia_pago'] = pagos_df['fecha_pago'].dt.day

pagos_df['mes_pago'].value_counts().sort_index().plot(kind='bar')
plt.title('Cantidad de Pagos por Mes')
plt.xlabel('Mes')
plt.ylabel('Cantidad de pagos')
plt.show()

sns.histplot(pagos_df['dia_pago'], bins=31, kde=False)
plt.title('Distribución de Pagos por Día del Mes')
plt.xlabel('Día del mes')
plt.ylabel('Cantidad de pagos')
plt.show()


In [ ]:

# --- Análisis de retrasos por perfil ---
sns.boxplot(data=pagos_df, x='perfil_pago', y='dias_retraso')
plt.title('Días de Retraso por Perfil de Pago')
plt.show()

sns.histplot(data=pagos_df, x='dias_retraso', hue='perfil_pago', multiple='stack')
plt.title('Distribución de Días de Retraso por Perfil')
plt.show()


In [ ]:

# --- Métodos de pago más utilizados (si la columna existe) ---
if 'metodo_pago' in pagos_df.columns:
    pagos_df['metodo_pago'].value_counts().plot(kind='bar')
    plt.title('Métodos de Pago Más Utilizados')
    plt.xlabel('Método de pago')
    plt.ylabel('Cantidad de pagos')
    plt.show()



## Conclusión

El EDA permitió identificar el perfil de pago de cada socio, analizar la frecuencia de pagos en el tiempo, los días de retraso y el uso de métodos de pago.
Esto sienta la base para estrategias de segmentación, optimización de precios y predicciones futuras.
